 DAEDALUS – Distributed and Automated Evolutionary Deep Architecture Learning with Unprecedented Scalability

This research code was developed as part of the research programme Open Technology Programme with project number 18373, which was financed by the Dutch Research Council (NWO), Elekta, and Ortec Logiqcare.

Project leaders: Peter A.N. Bosman, Tanja Alderliesten
Researchers: Alex Chebykin, Arthur Guijt, Vangelis Kostoulas
Main code developer: Arthur Guijt

In [1]:
import pandas as pd
from pathlib import Path

In [8]:
def set_run(df, idx):
    df["run"] = idx
    return df

gomea_elitists = pd.concat([
        set_run(pd.read_csv(p, sep=" "), idx)
        for idx, p in enumerate(Path("./worstofmaxcut_parallel__l_200__fns_2__GOMEA").glob("*/elitists.txt"))
    ])
gomea_elitists["approach"] = "GOMEA"
lkgomea_elitists = pd.concat([
        set_run(pd.read_csv(p, sep=" "), idx)
        for idx, p in enumerate(Path("./worstofmaxcut_parallel__l_200__fns_2__LKGOMEA_B").glob("*/elitists.txt"))
    ])
lkgomea_elitists["approach"] = "LKGOMEA"
results = pd.concat([
    gomea_elitists,
    lkgomea_elitists,
])

In [10]:
results

,#Evaluations,"Time,millisec.",Fitness,IsKey,PopulationSize,Solution,run,approach
0,1,0,29869.0,0,2,1110000010101001100010000010110011101011101011...,0,GOMEA
1,6,5,29998.0,0,4,1101010011110101011000010010110000111001111010...,0,GOMEA
2,64,24,30052.0,0,8,1100000111100110011100001101100010010001110000...,0,GOMEA
3,70,24,30095.0,0,8,1100000111100111011100001100000010010001110000...,0,GOMEA
4,72,24,30121.0,0,8,1100000111100111011100001100000010010001110000...,0,GOMEA
...,...,...,...,...,...,...,...,...
107,4337592,593092,30974.0,0,1024,0010001001111111111110111110001001000111001111...,19,LKGOMEA
108,4348875,593500,30977.0,0,1024,1010010011101101111001110110001011000011011101...,19,LKGOMEA
109,4923173,714055,30982.0,0,1024,1100000111000011111001111111001011001101111001...,19,LKGOMEA
110,7027557,1077277,30985.0,0,2048,1100010111000011111001111111001011001101111001...,19,LKGOMEA


In [14]:
from scipy.stats import mannwhitneyu

In [17]:
best_per_approach = results.groupby(["run", "approach"])["Fitness"].max().reset_index().groupby("approach")["Fitness"].apply(list).reset_index()

In [22]:
pairs = pd.merge(best_per_approach, best_per_approach, how="cross")
pairs = pairs[pairs["approach_x"] == pairs["approach_y"]]

pairs["test"] = pairs.apply((lambda r: mannwhitneyu(r["Fitness_x"], r["Fitness_y"], alternative="greater", use_continuity=False)), axis=1)
pairs

,approach_x,Fitness_x,approach_y,Fitness_y,test
0,GOMEA,"[30991.0, 30994.0, 30986.0, 30981.0, 30978.0, ...",GOMEA,"[30991.0, 30994.0, 30986.0, 30981.0, 30978.0, ...","(200.0, 0.5)"
3,LKGOMEA,"[30997.0, 30992.0, 30990.0, 30993.0, 30983.0, ...",LKGOMEA,"[30997.0, 30992.0, 30990.0, 30993.0, 30983.0, ...","(200.0, 0.5)"


In [25]:
best_per_approach

,approach,Fitness
0,GOMEA,"[30991.0, 30994.0, 30986.0, 30981.0, 30978.0, ..."
1,LKGOMEA,"[30997.0, 30992.0, 30990.0, 30993.0, 30983.0, ..."


In [27]:
best_per_approach.iloc[0]["Fitness"]

[30991.0,
 30994.0,
 30986.0,
 30981.0,
 30978.0,
 30991.0,
 30975.0,
 30963.0,
 30979.0,
 30996.0,
 30975.0,
 31002.0,
 30967.0,
 30993.0,
 30987.0,
 30973.0,
 31001.0,
 30992.0,
 30985.0,
 30985.0]

In [28]:
best_per_approach.iloc[1]["Fitness"]

[30997.0,
 30992.0,
 30990.0,
 30993.0,
 30983.0,
 30983.0,
 30999.0,
 30992.0,
 30991.0,
 30982.0,
 30993.0,
 30988.0,
 30988.0,
 31001.0,
 30995.0,
 30985.0,
 30988.0,
 30997.0,
 30986.0,
 30987.0]

In [31]:
results.iloc[results["Fitness"].argmax()]["Solution"]

'01011100010000001100011110010101010101001111011011111010010111100010011011111110110100001110010010100000110111111111100111110100001001011000100000100001100111110100111000000100100001011011001101101000'